In [1]:
%load_ext autoreload
%autoreload 2

# 1.数据预处理模块
**DataPreprocess.py**   
目前的数据主要为json格式.
需要进过多部处理才可以作为输入模型的数据.

1. 读取json文件
2. 转为为NodePlan
3. 转为为dataframe (添加两个字段: sql_id, parent_id)
4. 统计各个特征的分布等.数据统计


In [2]:
# 1.数据处理
import sys, os
sys.path.append(os.path.abspath(".."))  # 确保当前目录加入路径

from models.DataPreprocessor import get_plans_dict, DataPreprocessor, plan_trees_to_graphs, graphs_to_df, df_to_graphs, DFStatisticsInfo

json_path = "../data/train_plan_*.csv"
plans_dict, execution_times = get_plans_dict(json_path)
print("plans_dict:\n", plans_dict[0:5])
print("execution_times:\n", execution_times[0:5])

preprocessor = DataPreprocessor()
plans_tree = preprocessor.preprocess_all(plans_dict)

edges_list, matrix_plans = plan_trees_to_graphs(plans_tree, add_self_loops=True, undirected=False)
print(matrix_plans[0][0])
print(matrix_plans[0][1])
print(edges_list[0])
print(edges_list[99])

plans_df = graphs_to_df(matrix_plans)
plans_df.to_csv("../data/process/01_plans_df.csv", index=False)
plans_df.head()

# stats = DFStatisticsInfo(plans_df, sample_threshold=200, sample_k=10, strict_alias_check=True)
# node_types = stats.get_node_type_set()
# global_must = stats.global_must_keys()
# global_all  = stats.global_all_keys()
# per_key     = stats.per_key_values()
# per_type    = stats.per_nodetype_key_stats()
# issues      = stats.report_issues()
# stats.pretty_print_report()


找到的文件: ['../data/train_plan_part17.csv', '../data/train_plan_part8.csv', '../data/train_plan_part6.csv', '../data/train_plan_part3.csv', '../data/train_plan_part19.csv', '../data/train_plan_part9.csv', '../data/train_plan_part11.csv', '../data/train_plan_part1.csv', '../data/train_plan_part0.csv', '../data/train_plan_part18.csv', '../data/train_plan_part10.csv', '../data/train_plan_part12.csv', '../data/train_plan_part16.csv', '../data/train_plan_part15.csv', '../data/train_plan_part2.csv', '../data/train_plan_part14.csv', '../data/train_plan_part5.csv', '../data/train_plan_part7.csv', '../data/train_plan_part13.csv', '../data/train_plan_part4.csv']
总数据行数: 100000
df:
       id                                               json
0  85000  {"Plan": {"Node Type": "Bitmap Heap Scan", "Pa...
1  85001  {"Plan": {"Node Type": "Gather", "Parallel Awa...
2  85002  {"Plan": {"Node Type": "Hash Join", "Parallel ...
3  85003  {"Plan": {"Node Type": "Gather", "Parallel Awa...
4  85004  {"Plan": {"No

,plan_id,node_idx,Node Type,Parallel Aware,Relation Name,Alias,Startup Cost,Total Cost,Plan Rows,Plan Width,...,Peak Memory Usage,Filter,Rows Removed by Filter,Join Filter,Rows Removed by Join Filter,Merge Cond,Sort Key,Sort Method,Sort Space Used,Sort Space Type
0,0,0,Bitmap Heap Scan,False,movie_keyword,mk,11788.77,49094.94,1028173,12,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,1,Bitmap Index Scan,False,NaN,NaN,0.00,11531.73,1028173,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,0,Gather,False,NaN,NaN,59038.23,292179.55,958880,136,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,1,Hash Join,True,NaN,NaN,58038.23,195291.55,399533,136,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,2,Index Scan,True,cast_info,ci,0.44,96872.38,1832141,42,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
from models.DataPreprocessor import safe_cond_parse
import pandas as pd

NEED_PARSE_COND_SCAN = [
    "Filter",
    "Index Cond",
    "Recheck Cond"
]

NEED_PARSE_COND_JOIN = [
    "Hash Cond",
    "Join Filter",
    "Merge Cond",
]

NEED_PARSE_COND_COLS = NEED_PARSE_COND_SCAN + NEED_PARSE_COND_JOIN

for col in NEED_PARSE_COND_COLS:
    plans_df[f"{col}_Split"] = plans_df[col].apply(safe_cond_parse)


print(plans_df.head())
print("-"*100)
db_info = pd.read_csv("../data/column_min_max_vals.csv")
db_info["table_name"], db_info["column_name"] = db_info["name"].str.split(".").str[0], db_info["name"].str.split(".").str[1]
print(db_info.head())
print("-"*100)


graphs = df_to_graphs(plans_df)
print(graphs[0])
print("-"*100)





   plan_id  node_idx          Node Type  Parallel Aware  Relation Name Alias  \
0        0         0   Bitmap Heap Scan           False  movie_keyword    mk   
1        0         1  Bitmap Index Scan           False            NaN   NaN   
2        1         0             Gather           False            NaN   NaN   
3        1         1          Hash Join            True            NaN   NaN   
4        1         2         Index Scan            True      cast_info    ci   

   Startup Cost  Total Cost  Plan Rows  Plan Width  ...  Sort Key  \
0      11788.77    49094.94    1028173          12  ...       NaN   
1          0.00    11531.73    1028173           0  ...       NaN   
2      59038.23   292179.55     958880         136  ...       NaN   
3      58038.23   195291.55     399533         136  ...       NaN   
4          0.44    96872.38    1832141          42  ...       NaN   

   Sort Method  Sort Space Used  Sort Space Type Filter_Split  \
0          NaN              NaN        

# 2. 数据准备
TrainAndEval.py

## 该模块主要内容
1. 将数据划分为训练集,验证集,测试集
2. 构建数据集
3. 创建数据加载器



In [7]:
import torch
import numpy as np
from models.TrainAndEval import build_dataset
# 构建数据集
dataset = build_dataset(graphs, edges_list, execution_times)
print(f"数据集大小: {len(dataset)}")
for i in range(3):
    print(dataset[i].x)
    print(dataset[i].edge_index)
    print(dataset[i].y)
    # print(f"样本: x.shape={dataset[i].x.shape}, edge_index.shape={dataset[i].edge_index.shape}, y={dataset[i].y}")



数据集大小: 100000
[{'Node Type': 'Bitmap Heap Scan', 'Parallel Aware': False, 'Relation Name': 'movie_keyword', 'Alias': 'mk', 'Startup Cost': 11788.77, 'Total Cost': 49094.94, 'Plan Rows': 1028173, 'Plan Width': 12, 'Actual Startup Time': 41.924, 'Actual Total Time': 200.35, 'Actual Rows': 1029758, 'Actual Loops': 1, 'Recheck Cond': '(keyword_id > 17243)', 'Rows Removed by Index Recheck': 0.0, 'Exact Heap Blocks': 24037.0, 'Lossy Heap Blocks': 0.0, 'Parent Relationship': nan, 'Index Name': nan, 'Index Cond': nan, 'Workers Planned': nan, 'Workers Launched': nan, 'Single Copy': nan, 'Join Type': nan, 'Inner Unique': nan, 'Hash Cond': nan, 'Workers': nan, 'Scan Direction': nan, 'Hash Buckets': nan, 'Original Hash Buckets': nan, 'Hash Batches': nan, 'Original Hash Batches': nan, 'Peak Memory Usage': nan, 'Filter': nan, 'Rows Removed by Filter': nan, 'Join Filter': nan, 'Rows Removed by Join Filter': nan, 'Merge Cond': nan, 'Sort Key': nan, 'Sort Method': nan, 'Sort Space Used': nan, 'Sort Spa

In [9]:
# 5. 数据准备
from sklearn.model_selection import train_test_split
from torch_geometric.loader import DataLoader

# 构建数据集
dataset = build_dataset(graphs, edges_list, execution_times)
print(f"数据集大小: {len(dataset)}")
for i in range(3):
    print(dataset[i].x)
    print(dataset[i].edge_index)
    print(dataset[i].y)
    # print(f"样本: x.shape={dataset[i].x.shape}, edge_index.shape={dataset[i].edge_index.shape}, y={dataset[i].y}")
    
# 数据集划分
train_indices, temp_indices = train_test_split(
    range(len(dataset)), test_size=0.3, random_state=42
)
val_indices, test_indices = train_test_split(
    temp_indices, test_size=0.5, random_state=42
)

train_dataset = [dataset[i] for i in train_indices]
val_dataset = [dataset[i] for i in val_indices]
test_dataset = [dataset[i] for i in test_indices]

print(f"训练集: {len(train_dataset)}, 验证集: {len(val_dataset)}, 测试集: {len(test_dataset)}")

# 创建数据加载器
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

print(f"训练批次数: {len(train_loader)}, 验证批次数: {len(val_loader)}")



数据集大小: 100000
[{'Node Type': 'Bitmap Heap Scan', 'Parallel Aware': False, 'Relation Name': 'movie_keyword', 'Alias': 'mk', 'Startup Cost': 11788.77, 'Total Cost': 49094.94, 'Plan Rows': 1028173, 'Plan Width': 12, 'Actual Startup Time': 41.924, 'Actual Total Time': 200.35, 'Actual Rows': 1029758, 'Actual Loops': 1, 'Recheck Cond': '(keyword_id > 17243)', 'Rows Removed by Index Recheck': 0.0, 'Exact Heap Blocks': 24037.0, 'Lossy Heap Blocks': 0.0, 'Parent Relationship': nan, 'Index Name': nan, 'Index Cond': nan, 'Workers Planned': nan, 'Workers Launched': nan, 'Single Copy': nan, 'Join Type': nan, 'Inner Unique': nan, 'Hash Cond': nan, 'Workers': nan, 'Scan Direction': nan, 'Hash Buckets': nan, 'Original Hash Buckets': nan, 'Hash Batches': nan, 'Original Hash Batches': nan, 'Peak Memory Usage': nan, 'Filter': nan, 'Rows Removed by Filter': nan, 'Join Filter': nan, 'Rows Removed by Join Filter': nan, 'Merge Cond': nan, 'Sort Key': nan, 'Sort Method': nan, 'Sort Space Used': nan, 'Sort Spa

# 3. 模型模块

## NodeEncoder
NodeEncoder.py
主要包括了各种vectorical的编码方式.

1. 转为为Matrix(Node, Edge)


## TreeEncoder
TreeEncoder.py
目前包括两种模型一个是GAT,一个是传统GNN.
1. 转为vector

## PredictionHead
PredictionHead.py
目前进行最简单的FNN进行后续回归任务

1. 预测






In [10]:
# 模型搭建

import torch
import torch.nn as nn
from torch_geometric.data import Data, Batch

from models.NodeEncoder import *
from models.TreeEncoder import *
from models.PredictionHead import *
from models.TrainAndEval import EarlyStopping

class CostModel(nn.Module):
    """
    NodeEncoder → GATTreeEncoder → PredictionHead
    """
    def __init__(self, nodecoder: nn.Module, treeencoder: nn.Module, predict_head: nn.Module):
        super().__init__()
        self.nodecoder = nodecoder
        self.treeencoder = treeencoder
        self.predict_head = predict_head

    def forward(self, data: Data | Batch):
        """
        data 结构
        - x: [N, F_num]
        - edge_index: [2, E]
        """
        x = self.nodecoder(data.x)                                   # [N, d_node]
        g = self.treeencoder(x, data.edge_index, data.batch)         # [B, d_graph]
        y = self.predict_head(g)                                     # [B, out_dim]
        return y

f_in, d_node, d_graph, out_dim = 16, 32, 64, 1
nodecoder = NodeEncoder_Mini(
    in_dim=f_in,
    d_node=d_node
)
gatTreeEncoder = TreeEncoder_GATMini(
    in_dim=d_node,
    hidden_dim=64,
    out_dim=d_graph
)
predict_head = PredictionHead_FNNMini(
    in_dim=d_graph,
    out_dim=out_dim
)

# 模型构建
model = CostModel(nodecoder, gatTreeEncoder, predict_head)
print(model)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"使用设备: {device}")
model = model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=10, factor=0.5)
criterion = torch.nn.MSELoss()
early_stopping = EarlyStopping(patience=15, min_delta=0.001)

CostModel(
  (nodecoder): NodeEncoder_Mini(
    (proj): Sequential(
      (0): Linear(in_features=16, out_features=32, bias=True)
      (1): ReLU()
      (2): LayerNorm((32,), eps=1e-05, elementwise_affine=True)
    )
  )
  (treeencoder): TreeEncoder_GATMini(
    (gat1): GATConv(32, 64, heads=8)
    (gat2): GATConv(512, 64, heads=1)
  )
  (predict_head): PredictionHead_FNNMini(
    (mlp): Sequential(
      (0): Linear(in_features=64, out_features=128, bias=True)
      (1): ReLU()
      (2): Dropout(p=0.1, inplace=False)
      (3): Linear(in_features=128, out_features=64, bias=True)
      (4): ReLU()
      (5): Dropout(p=0.1, inplace=False)
      (6): Linear(in_features=64, out_features=1, bias=True)
    )
  )
)
使用设备: cuda


# 4 训练&评估
## 训练
主要模块为划分训练集,测试集,验证集.
调用模型进行训练.
## 评估
主要为评估方式.目前为MSE以及Q-error.


In [11]:
from datetime import datetime
from models.TrainAndEval import train_epoch, validate_epoch
import time

#  8. 训练循环
def train_model(model, train_loader, val_loader, optimizer, scheduler, criterion, 
                early_stopping, device, weight_path, num_epochs=100):
    
    train_losses = []
    val_losses = []
    best_val_loss = float('inf')
    
    print("开始训练...")
    print("-" * 60)
    
    for epoch in range(num_epochs):
        start_time = time.time()
        
        # 训练
        train_loss = train_epoch(model, train_loader, optimizer, criterion, device)
        
        # 验证
        val_loss = validate_epoch(model, val_loader, criterion, device)
        
        # 学习率调度
        scheduler.step(val_loss)
        
        # 记录损失
        train_losses.append(train_loss)
        val_losses.append(val_loss)
        
        # 计算时间
        epoch_time = time.time() - start_time
        
        # 打印进度
        if epoch % 5 == 0 or epoch == num_epochs - 1:
            print(f"Epoch {epoch+1:3d}/{num_epochs} | "
                  f"Train Loss: {train_loss:.6f} | "
                  f"Val Loss: {val_loss:.6f} | "
                  f"LR: {optimizer.param_groups[0]['lr']:.2e} | "
                  f"Time: {epoch_time:.2f}s")
        
        # 早停检查
        if early_stopping(val_loss, model):
            print(f"\n早停触发在第 {epoch+1} 轮")
            break
        
        # 保存最佳模型
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            date = datetime.now().strftime("%m%d")
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'train_loss': train_loss,
                'val_loss': val_loss,
            }, f'../results/weight_{date}.pth')
    
    print("-" * 60)
    print("训练完成!")
    print(f"最佳验证损失: {best_val_loss:.6f}")
    
    return train_losses, val_losses

# 开始训练
date = datetime.now().strftime("%m%d")
weight_path = f'../results/{date}.pth'
train_losses, val_losses = train_model(
    model, train_loader, val_loader, optimizer, scheduler, 
    criterion, early_stopping, device, weight_path, num_epochs=100
)


开始训练...
------------------------------------------------------------


/home/AiChaosN/miniconda3/envs/cs224/lib/python3.12/site-packages/torch_geometric/data/storage.py:452: UserWarning: Unable to accurately infer 'num_nodes' from the attribute set '{'edge_index', 'x', 'y'}'. Please explicitly set 'num_nodes' as an attribute of 'data' to suppress this warning
  warnings.warn(


TypeError: linear(): argument 'input' (position 1) must be Tensor, not list

In [12]:

# 10. 可视化训练过程和结果
import matplotlib.pyplot as plt

def plot_training_history(train_losses, val_losses):
    plt.figure(figsize=(12, 4))
    
    # 训练损失曲线
    plt.subplot(1, 2, 1)
    epochs = range(1, len(train_losses) + 1)
    plt.plot(epochs, train_losses, 'b-', label='Train Loss')
    plt.plot(epochs, val_losses, 'r-', label='Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Training and Validation Loss')
    plt.legend()
    plt.grid(True)
    
    # 预测 vs 真实值
    plt.subplot(1, 2, 2)
    plt.scatter(targets, predictions, alpha=0.5, s=20)
    min_val = min(targets.min(), predictions.min())
    max_val = max(targets.max(), predictions.max())
    plt.plot([min_val, max_val], [min_val, max_val], 'r--', lw=2)
    plt.xlabel('True Execution Time')
    plt.ylabel('Predicted Execution Time')
    plt.title('Predicted vs True Execution Time')
    plt.grid(True)
    
    plt.tight_layout()
    date = datetime.now().strftime("%m%d")
    plt.savefig(f'../results/training_results_{date}.png', dpi=300, bbox_inches='tight')
    plt.show()

# 创建结果目录
os.makedirs('../results', exist_ok=True)
# 绘制结果
plot_training_history(train_losses, val_losses)


NameError: name 'train_losses' is not defined

$$
x = [
\underbrace{x_1, ..., x_6,\ }_{\text{categorical}}
\underbrace{x_7, x_8,\ }_{\text{numeric}}
\underbrace{x_9, ..., x_{16},\ }_{\text{categorical}}
\underbrace{x_{17}, ..., x_{32}}_{\text{conditional}}
]
$$

$$
\tilde{x} = [
\underbrace{\tilde{x}_1, ..., \tilde{x}_6,\ }_{\text{NodeType}}
\underbrace{\tilde{x}_7, \tilde{x}_8,\ }_{\text{Size}}
\underbrace{\tilde{x}_9, ..., \tilde{x}_{16},\ }_{\text{Table}}
\underbrace{\tilde{x}_{17}, ..., \tilde{x}_{32}}_{\text{Predicate}}
]
$$
